# Portfolio Optimization

In [1]:
import cvxpy as cp
import numpy as np
import numpy.random as npr
import torch
import pandas as pd
import lropt

### Import Dataset and choose random stocks

In [2]:
sp500 = pd.read_csv('stock_data/prices_sp500.csv').to_numpy()
sp500_synthetic_returns = pd.read_csv('stock_data/sp500_synthetic_returns.csv').to_numpy()[:,1:]

num_stocks = 100
num_rets = 500

In [3]:
npr.seed(1)
stock_idxs = np.random.choice(sp500.shape[1], num_stocks, replace=False)

sp_prices = sp500[:num_rets+1, stock_idxs]
sp_rets = (sp_prices[1:,:] - sp_prices[:-1,:])/sp_prices[1:,:]

In [4]:
sp_rets.shape

(500, 100)

### Create LROPT problem

In [5]:
def violation_loss(t_soln, x_soln, data, lmbda=1):
    return t_soln + lmbda * torch.mean(
        torch.maximum(-data @ x_soln - t_soln, torch.tensor(0., requires_grad=True))), t_soln

unc_set = lropt.Ellipsoidal(data=sp_rets, loss=violation_loss)
u = lropt.UncertainParameter(num_stocks, uncertainty_set=unc_set)

x = cp.Variable(num_stocks)
t = cp.Variable()

objective = cp.Minimize(t)
cons = [-u @ x <= t]
cons += [cp.sum(x) == 1, x >= 0]

prob_robust = lropt.RobustProblem(objective, cons)
df = prob_robust.train()
prob_robust.solve()

/Users/irina.wang/Desktop/Princeton/Project2/cvxpy/cvxpy/expressions/expression.py:607: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 1 times so far.

  warnings.warn(msg, UserWarning)
/Users/irina.wang/Desktop/Princeton/Project2/cvxpy/cvxpy/expressions/expression.py:607: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 2 times so far.

  warnings.warn(msg, UserWarning)
/Use

-0.08372198960845363

torch.Size([100, 10])